In [12]:
import importlib
import json
import os
import pickle
import sys
import h3

from urllib.request import urlopen

import contextily as ctx
import folium
import geopandas as gpd
import matplotlib
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import pandas as pd
import geopandas as gpd
import rasterio
import shapely

# from h3 import h3
from mpl_toolkits.axes_grid1 import make_axes_locatable
from rasterio.crs import CRS
from shapely.geometry import Polygon, mapping, shape
from shapely.ops import unary_union
from tqdm import tqdm

h3_lvl = 6 
return_gdf=True
buffer0=False
read_pickle=False
pickle_file=f"h0_dictionary_of_h{h3_lvl}_geodata_frames.pickle"
pickle_file_land=f"h0_dictionary_of_h{h3_lvl}_geodata_frames_land.pickle"

In [ ]:
# Get list of all h3 lvl 0 cells
h3_lvl0 = list(h3.get_res0_cells())

# Generate list of all children of h3 lvl 0 cells
h3_lvl0_children = {}
for h3_0 in h3_lvl0:
    h3_children = list(h3.cell_to_children(h3_0, h3_lvl))
    if return_gdf:
        hex_poly = lambda hex_id: shape(
            h3.cells_to_geo([hex_id])
        ).buffer(0)
        all_polys = gpd.GeoSeries(
            list(map(hex_poly, h3_children)), index=h3_children, crs=4326
        )
        all_polys = gpd.GeoDataFrame(all_polys, crs=4326, columns=["geometry"])
        if buffer0:
            all_polys["geometry"] = all_polys["geometry"].apply(
                lambda x: x.buffer(0)
            )
        all_polys["shape_id"] = list(all_polys.index)

        h3_lvl0_children[h3_0] = all_polys
    else:
        h3_lvl0_children[h3_0] = h3_children

In [2]:
full_pickle = os.path.join("../../src", pickle_file)
h3_lvl0_children = pickle.load(open(full_pickle, "rb"))

In [3]:
# Loop through geo data frames and eliminate all cells that do not intersect land
world_filepath = r"C:\WBG\Work\data\ADMIN\ne_110m_admin_0_countries.shp"
world = gpd.read_file(world_filepath)
world['geometry'] = world['geometry'].apply(lambda x: x.buffer(0))
world_bounds = world.union_all()
world_gdf = gpd.GeoDataFrame(pd.DataFrame([[0, world_bounds]], columns=["gid", "geometry"]), crs=4326, geometry="geometry")

In [14]:
h3_lvl0_land = {}
for key, cur_df in h3_lvl0_children.items():
    cnt = cur_df.shape[0]
    sel_df = gpd.sjoin(cur_df, world, how='inner', predicate='intersects')
    sel_df = sel_df.loc[:, cur_df.columns]
    final_df = sel_df.loc[sel_df.index.drop_duplicates()]
    if final_df.shape[0] > 0:
        h3_lvl0_land[key] = final_df.copy()
    print(f"{key} has {final_df.shape[0]} of {cnt}")
    

8001fffffffffff has 10516 of 117649
8003fffffffffff has 37740 of 117649
8005fffffffffff has 51141 of 117649
8007fffffffffff has 51656 of 117649
8009fffffffffff has 39466 of 98041
800bfffffffffff has 114016 of 117649
800dfffffffffff has 59002 of 117649
800ffffffffffff has 69309 of 117649
8011fffffffffff has 110447 of 117649
8013fffffffffff has 111635 of 117649
8015fffffffffff has 109233 of 117649
8017fffffffffff has 30219 of 117649
8019fffffffffff has 18011 of 117649
801bfffffffffff has 9786 of 117649
801dfffffffffff has 44 of 98041
801ffffffffffff has 99411 of 117649
8021fffffffffff has 120228 of 117649
8023fffffffffff has 2973 of 117649
8025fffffffffff has 118630 of 117649
8027fffffffffff has 118168 of 117649
8029fffffffffff has 46609 of 117649
802bfffffffffff has 47178 of 117649
802dfffffffffff has 97535 of 117649
802ffffffffffff has 19039 of 117649
8031fffffffffff has 68036 of 98041
8033fffffffffff has 5043 of 117649
8035fffffffffff has 448 of 117649
8037fffffffffff has 0 of 117649


In [15]:
import pickle

with open(pickle_file_land, "wb") as f:
    pickle.dump(h3_lvl0_land, f)


# DEBURRGGLING

In [ ]:
h3_0_cells = h3.get_res0_cells()
h3_geoms = [Polygon(h3.cell_to_boundary(x)) for x in h3_0_cells]
all_h3_lvl0_geoms = gpd.GeoDataFrame(pd.DataFrame({'h3_id': h3_0_cells, 'geometry': h3_geoms}), geometry = 'geometry', crs = 4326)
all_h3_lvl0_geoms.to_file('C:/WBG/Work/S2S/all_lvl0_gems.geojson', driver='GeoJSON')